In [14]:
import multiprocessing
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import netCDF4 as cdf
from datetime import datetime, timedelta
from scipy.interpolate import griddata
import concurrent.futures

In [15]:
year = "2016"
explotion_times_directory = "explotion-times/"
nc_directory = "carra-files/param_130.nc"

## Open and read .nc file variables


In [16]:
nc = cdf.Dataset(nc_directory)
longitudes = nc.variables["longitude"][100:475, 477:]
latitudes = nc.variables["latitude"][100:475, 477:]
raw_times = nc.variables["time"][:]
temperture = nc.variables["t"][..., 100:475, 477:]
nc.close()

print(np.shape(longitudes))
print(np.shape(latitudes))
print(np.shape(raw_times))
print(np.shape(temperture))

(375, 312)
(375, 312)
(70,)
(70, 23, 375, 312)


## Organize data


In [17]:
new_longitudes = np.where(longitudes <= 180, longitudes, longitudes - 360)
reshaped_longitudes = new_longitudes.view().reshape(np.size(longitudes), 1)
reshaped_latitudes = latitudes.view().reshape(np.size(latitudes), 1)
coords_full_set = np.hstack((reshaped_longitudes, reshaped_latitudes))

longitudes_to_interp = np.linspace(25.5, 25, 72).reshape(72, 1)
latitudes_to_interp = np.linspace(66, 69, 72).reshape(72, 1)
coords_to_interp = np.block([longitudes_to_interp, latitudes_to_interp])


print(np.shape(longitudes_to_interp))
print(np.shape(latitudes_to_interp))
print(np.shape(coords_to_interp))

(72, 1)
(72, 1)
(72, 2)


In [18]:
arc_files = [
    filename
    for filename in os.listdir(explotion_times_directory)
    if filename[0:4] == year
]

print(arc_files)


whole_dates = np.array(  ## .nc  dates
    [
        datetime.utcfromtimestamp(posix_timestamp).strftime("%Y-%m-%dT%H:%M")
        for posix_timestamp in raw_times
    ],
    dtype="datetime64[m]",
)

explosion_dates = np.empty(len(arc_files), dtype="datetime64[m]")  # .arc dates


# iterar por cada .ARC y encontrar cuando exploto la bomba y sacar la fecha y salvarla a explosion_dates
for index, file in enumerate(arc_files):
    data = pd.read_csv(
        explotion_times_directory + file, delim_whitespace=True, header=None
    )

    # arc column where bomb exploded (eg: 366)
    explotion = np.argmax(data.iloc[:, 3])

    # arc column where bomb exploded (eg: 366)
    date_of_explotion = data.iloc[explotion, 0]

    year_of_explotion = date_of_explotion[0:4]
    day_number_of_explotion = date_of_explotion[5:8]
    hour_of_explotion = date_of_explotion[9:11]
    minutes_of_explotion = date_of_explotion[12:14]

    day_number_of_explotion = day_number_of_explotion.rjust(
        3 + len(day_number_of_explotion), "0"
    )
    template_date = datetime(
        int(year_of_explotion), 1, 1, int(hour_of_explotion), int(minutes_of_explotion)
    )
    resulting_object_date = template_date + timedelta(
        days=int(day_number_of_explotion) - 1
    )
    resulting_string_date = resulting_object_date.strftime("%Y-%m-%dT%H:%M")

    explosion_dates[index] = resulting_string_date

['2016-231_12.29.58.500.ARC', '2016-232_11.29.59.350.ARC', '2016-233_13.29.59.550.ARC', '2016-234_13.00.00.100.ARC', '2016-235_11.59.59.500.ARC', '2016-236_12.59.59.450.ARC', '2016-237_11.59.59.400.ARC', '2016-238_11.29.59.400.ARC', '2016-239_11.29.58.650.ARC', '2016-240_12.59.59.100.ARC', '2016-241_10.59.59.000.ARC', '2016-242_09.59.58.550.ARC', '2016-242_14.09.58.450.ARC', '2016-243_07.54.57.750.ARC', '2016-243_10.04.57.300.ARC', '2016-243_12.24.56.850.ARC', '2016-244_08.49.58.950.ARC', '2016-244_11.44.58.700.ARC', '2016-244_15.44.58.750.ARC']


In [19]:
# clean whole_dates to only keep the ones that match with explosion dates
def date_cleaner(whole_dates, explosion_dates):
    def index_finder(datetime_array, explosion_date):
        indices = np.argsort(np.abs(datetime_array - explosion_date))[:2]
        return indices

    desired_indices = []
    with np.nditer([explosion_dates], flags=["buffered"], op_flags=["readonly"]) as it:
        for explosion_date in it:
            desired_indices.extend(index_finder(whole_dates, explosion_date))

    return desired_indices


desired_indices = np.unique(
    date_cleaner(whole_dates, explosion_dates)
)  # indexes of cleaned dates
cleaned_dates = whole_dates[desired_indices]

In [20]:
def first_interpolator(kind_of_measurement, selected_time, out=None):
    # kind of mesurments is a num from 1 to 3
    # selected time is cleaned_dates

    def second_interpolator(sequence_of_interpolants):
        # sequence_of_interpolants is a list of lists

        # convert to array
        array_of_interpolants = np.array(sequence_of_interpolants)
        # shape(heigths, times, interpolants)

        index = 0
        with np.nditer(
            [array_of_interpolants, out],
            flags=["buffered", "multi_index", "refs_ok"],
            op_flags=[["readonly"], ["writeonly", "allocate", "no_broadcast"]],
        ) as it:
            for measurement_values, interpolants_for_a_time in it:
                # interpolant is what the program outputs
                interpolant = np.interp(
                    explosion_dates,
                    cleaned_dates,
                    [
                        measurement_values[:, index, :],
                        measurement_values[:, index + 1, :],
                    ],
                )
                interpolants_for_a_time[...] = interpolant
                index += 1
                result = it.operands[1]
        print(result)

        print("\nInterpolación temporal exitosa\n")

    if kind_of_measurement == 1:
        cleaned_temperture = temperture[
            desired_indices, ...
        ]  # get temperatures we want with indexes

        # clean temps is a 4d array

        """
        example = [
            [
                [
                    [
                        []#y size ~250
                    ]#x size ~250
                    ]#height size 23
            ] #time size 33 
        ]
        """

        # loop that
        with np.nditer(
            [np.arange(0, 1, 1)]
        ) as it:  # en teoría aquí va un 23, por 23 alturas
            for height in it:
                # each iterations appends a list to this variable
                measurements_at_same_height = (
                    []
                )  # aquí sería un arreglo creado con np.empty
                with np.nditer(
                    [np.arange(0, 2, 1)]
                ) as it:  # aquí el tamaño del arreglo cleaned dates
                    for time in it:
                        print(f"time: {time}")  # int ,0,1,2...
                        print(f"height: {height}")  # int ,0,0..int

                        flatten_temperture = cleaned_temperture[
                            time, height, ...
                        ].ravel()
                        temperture_interpolants = griddata(
                            coords_full_set,
                            flatten_temperture,
                            coords_to_interp,
                            method="cubic",
                        )
                        measurements_at_same_height.append(
                            list(temperture_interpolants)
                        )
                        # measurements_at_same_height[index] = list(temperture_interpolants) en caso de usar el arreglo creado con np.empty
                sequence_of_interpolants.append(measurements_at_same_height)

    print("Interpolación espacial exitosa")
    second_interpolator(sequence_of_interpolants)


sequence_of_interpolants = []

kind_of_measurement = int(
    input(
        "1. Temperatura\n2. Velocidad del viento\n3. Dirección del viento\nIntroduce el número de la medida a interpolar: "
    )
)
while kind_of_measurement in range(1, 4):
    break
    print("\nInterpolación en proceso...\n")
    first_interpolator(kind_of_measurement, cleaned_dates)
    kind_of_measurement = False

In [21]:
import concurrent.futures
import time


def nested_loop():
    cleaned_temperture = temperture[desired_indices, ...]

    def process_griddata(time_i, height_i):
        return list(
            griddata(
                coords_full_set,
                cleaned_temperture[time_i, height_i, ...].ravel(),
                coords_to_interp,
                method="cubic",
            )
        )

    measurements_at_same_height = []

    # Create a ThreadPoolExecutor with 3 threads
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        # Iterate over the outer loop
        for time_i in range(np.shape(cleaned_temperture)[0]):
            # Submit each iteration of the inner loop as a separate task to the executor
            futures = [
                executor.submit(process_griddata, time_i, height_i)
                for height_i in range(np.shape(cleaned_temperture)[1])
            ]

            # Wait for all tasks to complete
            concurrent.futures.wait(futures)

            # Process the results
            results = [future.result() for future in futures]
            measurements_at_same_height.extend(results)


def interpolate(measurement, dates):
    sequence_of_interpolants = []

    if measurement == 1:
        # get temperatures we want with indexes
        cleaned_temperture = temperture[desired_indices, ...]

        # clean temps is a 4d array

        """
        example = [
            [
                [
                    [
                        []#y size ~250
                    ]#x size ~250
                    ]#height size 23
            ] #time size 33 
        ]
        """
        measurements_at_same_height = []

        for time_i in tqdm(range(np.shape(cleaned_temperture)[0]), desc="Loading..."):
            for height_i in range(np.shape(cleaned_temperture)[1]):
                measurements_at_same_height.append(
                    list(
                        griddata(
                            coords_full_set,
                            cleaned_temperture[time_i, height_i, ...].ravel(),
                            coords_to_interp,
                            method="cubic",
                        )
                    )
                )

                """
                clear_output(wait=True)
                print(
                    f"Progress: {(((time_i * 10) + height_i + 1 ) / (np.shape(cleaned_temperture)[0]*np.shape(cleaned_temperture)[1])) * 100}%"
                )

                
        
                for x_i in range(np.shape(cleaned_temperture)[2]):
                    for y_i in range(np.shape(cleaned_temperture)[3]):
                        pass
                        #print(x)
                """
        print(measurements_at_same_height)


# interpolate(1, cleaned_dates)

In [22]:
import concurrent.futures
import time


def process_item(i, j):
    time.sleep(1)
    return (i, j)


def nested_loop():
    l = []

    # Create a ThreadPoolExecutor with 3 threads
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        # Iterate over the outer loop
        for i in range(10):
            # Submit each iteration of the inner loop as a separate task to the executor
            futures = [executor.submit(process_item, i, j) for j in range(20)]

            # Wait for all tasks to complete
            concurrent.futures.wait(futures)

            # Process the results
            results = [future.result() for future in futures]
            l.extend(results)

    print(l)

In [32]:
def interpolate():
    cleaned_temperture = temperture[desired_indices, ...]

    def process_griddata(time_i, height_i):
        return list(
            griddata(
                coords_full_set,
                cleaned_temperture[time_i, height_i, ...].ravel(),
                coords_to_interp,
                method="cubic",
            )
        )

    array_of_interpolants = []

    # Create a ThreadPoolExecutor with max threads
    with concurrent.futures.ThreadPoolExecutor(
        max_workers=multiprocessing.cpu_count()
    ) as executor:
        progress_bar = tqdm(
            total=np.shape(cleaned_temperture)[0] * np.shape(cleaned_temperture)[1]
        )

        # Iterate over the outer loop
        for time_i in range(np.shape(cleaned_temperture)[0]):
            # Submit each iteration of the inner loop as a separate task to the executor
            futures = [
                executor.submit(process_griddata, time_i, height_i)
                for height_i in range(np.shape(cleaned_temperture)[1])
            ]

            # Update the progress bar for each submitted task
            for future in futures:
                future.add_done_callback(lambda p: progress_bar.update(1))

            # Wait for all tasks to complete
            concurrent.futures.wait(futures)

            # Process the results
            array_of_interpolants = np.array([future.result() for future in futures])

    print(array_of_interpolants)
    np.save("array_of_interpolants.npy", array_of_interpolants)


interpolate()

100%|██████████| 759/759 [08:40<00:00,  1.46it/s]

[[285.27858405 285.22968264 285.218812   ... 283.25728675 283.26878657
  283.29251187]
 [283.27811038 283.249422   283.25590294 ... 280.50842362 280.51865924
  280.5414514 ]
 [281.99334551 282.072912   282.16131737 ... 279.41696986 279.39569965
  279.41756179]
 ...
 [222.56860797 222.57062655 222.57335093 ... 223.26464907 223.26992991
  223.27742676]
 [218.5185453  218.51811619 218.51812988 ... 218.80826515 218.80990748
  218.81333902]
 [215.45677907 215.45382782 215.45117549 ... 215.26340738 215.26156835
  215.26085135]]


In [33]:
array_of_interpolants = np.load("array_of_interpolants.npy")
print(array_of_interpolants)

result = []

for i, measurement_values in enumerate(array_of_interpolants):
    interpolant = np.interp(
        explosion_dates,
        cleaned_dates,
        [
            measurement_values[:, i, :],
            measurement_values[:, i + 1, :],
        ],
    )
    result.append(interpolant)


result = np.array(result)


print(result)

[[285.27858405 285.22968264 285.218812   ... 283.25728675 283.26878657
  283.29251187]
 [283.27811038 283.249422   283.25590294 ... 280.50842362 280.51865924
  280.5414514 ]
 [281.99334551 282.072912   282.16131737 ... 279.41696986 279.39569965
  279.41756179]
 ...
 [222.56860797 222.57062655 222.57335093 ... 223.26464907 223.26992991
  223.27742676]
 [218.5185453  218.51811619 218.51812988 ... 218.80826515 218.80990748
  218.81333902]
 [215.45677907 215.45382782 215.45117549 ... 215.26340738 215.26156835
  215.26085135]]


IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [27]:
np.load("array_of_interpolants.npy")

array([1, 2, 3])

In [31]:
array_of_interpolants = np.array([i for i in range(58)])
array_of_interpolants

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57])